In [465]:
import numpy as np
import tensorflow as tf
import re
import time
import pandas as pd

IMPORT THE DATASET

In [466]:
lines = open('datasets/WikiQACorpus/WikiQACorpus/WikiQA-test.txt', encoding='utf-8', errors = 'ignore').read().split('.')
conversations = open('datasets/WikiQACorpus/WikiQACorpus/WikiQA-train.txt', encoding='utf-8', errors = 'ignore').read().split('.')

In [467]:
print(len(lines))
print(len(conversations))

6993
22807


In [468]:
lines

['HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tAfrican immigration to the United States refers to immigrants to the United States who are or were nationals of Africa ',
 '\t0\nHOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tThe term African in the scope of this article refers to geographical or national origins rather than racial affiliation ',
 '\t0\nHOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tFrom the Immigration and Nationality Act of 1965 to 2007 , an estimated total of 0',
 '8 to 0',
 '9 million Africans immigrated to the United States , accounting for roughly 3',
 '3 % of total immigration to the United States during this period ',
 '\t0\nHOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tAfrican immigrants in the United States come from almost all regions in Africa and do not constitute a homogeneous group ',
 '\t0\nHOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tThey include people from different national , linguistic , ethnic , racial , cultural and social backgrounds

Creating a dictionary that maps each line and its id

In [469]:
id2line = []
# print(lines[:10])
# print()
for line in lines:
    _line = line.split('\t0\n')
    
    id2line.append(_line)

# print(id2line[:10])
# print()


print(len(id2line))
print(id2line)

6993
[['HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tAfrican immigration to the United States refers to immigrants to the United States who are or were nationals of Africa '], ['', 'HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tThe term African in the scope of this article refers to geographical or national origins rather than racial affiliation '], ['', 'HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tFrom the Immigration and Nationality Act of 1965 to 2007 , an estimated total of 0'], ['8 to 0'], ['9 million Africans immigrated to the United States , accounting for roughly 3'], ['3 % of total immigration to the United States during this period '], ['', 'HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tAfrican immigrants in the United States come from almost all regions in Africa and do not constitute a homogeneous group '], ['', 'HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US\tThey include people from different national , linguistic , ethnic , racial , cultural and social ba

In [470]:
id2line = id2line[:601]
count = 0 

question = []
answer = []


for line in id2line :
    line.pop(0)
    if len(line) != 0 :
#         print(line)
        temp_lines = line[0].split('\t')

#         print(temp_lines)        
        print("{}question:{}".format(count,temp_lines[0]))
        print("answer:",temp_lines[1])
#         print(type(temp_lines[0]))
        
        question.append(str(temp_lines[0]))
        if isinstance(temp_lines[1],str) :
            answer.append(str(temp_lines[1]))
        else :
            answer.append('')

        count += 1
        
print ("question:",question[:10])
print ("answer:",answer[:10])
        
        

0question:HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
answer: The term African in the scope of this article refers to geographical or national origins rather than racial affiliation 
1question:HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
answer: From the Immigration and Nationality Act of 1965 to 2007 , an estimated total of 0
2question:HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
answer: African immigrants in the United States come from almost all regions in Africa and do not constitute a homogeneous group 
3question:HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
answer: They include people from different national , linguistic , ethnic , racial , cultural and social backgrounds 
4question:HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
answer: As such , African immigrants are to be distinguished from African American people , the latter of whom are descendants of mostly West and Central Africans who were involuntarily brought to the United States by means of the historic At

creating a list of all of the conversation

Getting swparately the questions and the answers

In [471]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'l'", "will", text)
    text = re.sub(r"\'ve'", "have", text)
    text = re.sub(r"\d", "would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"/@;'<>'.?,|]", "", text)
    return text

cleaning the question and answers

In [472]:
clean_questions = []
for CleanQuestion in question:
    clean_questions.append(clean_text(CleanQuestion))
    
clean_answers = []
for CleanAnswer in answer:
    clean_answers.append(clean_text(CleanAnswer))

print("Cleaned_Question:", clean_questions)
print()
print()
print()
print("Cleaned_Answer:", clean_answers)


Cleaned_Question: ['how african americans were immigrated to the us', 'how african americans were immigrated to the us', 'how african americans were immigrated to the us', 'how african americans were immigrated to the us', 'how african americans were immigrated to the us', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how large were early jails', 'how a water pump works', 'how a water pump works', 'how a water pump works', 'how a water pump works', 'how old was sue lyon when she made lolita', 'how old was sue lyon when she made lolita', 'how old was sue lyon when she made lolita', 'how old was sue lyon when she made lolita', 'how are antibodies used in', 'how are antibodies used in', 'how are antibodies used in', 'how are antibodies used in', 'how are antibodies us

Creating a dictionary that maps each word to its number of occurences

In [473]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [474]:
word2count

{'how': 388,
 'african': 10,
 'americans': 6,
 'were': 29,
 'immigrated': 5,
 'to': 261,
 'the': 842,
 'us': 10,
 'large': 16,
 'early': 18,
 'jails': 10,
 'a': 327,
 'water': 11,
 'pump': 6,
 'works': 5,
 'old': 32,
 'was': 99,
 'sue': 6,
 'lyon': 6,
 'when': 21,
 'she': 30,
 'made': 17,
 'lolita': 7,
 'are': 158,
 'antibodies': 28,
 'used': 39,
 'in': 346,
 'is': 358,
 'alicia': 25,
 'wouldwouldwouldwould': 135,
 'big': 17,
 'land': 19,
 'section': 24,
 'generally': 19,
 'shakespeare': 6,
 's': 65,
 'juliet': 7,
 'fire': 8,
 'bricks': 10,
 'much': 32,
 'centavos': 8,
 'mexico': 8,
 'long': 22,
 'mickie': 10,
 'james': 17,
 'with': 99,
 'wwe': 13,
 'can': 55,
 'something': 18,
 'taste': 35,
 'good': 18,
 'one': 44,
 'person': 26,
 'did': 76,
 'bob': 9,
 'marley': 11,
 'die': 16,
 'production': 7,
 'of': 452,
 'willy': 9,
 'wonka': 9,
 'and': 354,
 'chocolate': 10,
 'factory': 9,
 'ream': 3,
 'paper': 5,
 'i': 19,
 'open': 26,
 'usda': 13,
 'slaughterhouse': 14,
 'close': 17,
 'or': 11

creating two dictionaries that map the questions words and the answers words to unique interger

In [475]:
threshold_questions = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1


In [476]:
questionsword2int

{'how': 0,
 'were': 1,
 'to': 2,
 'the': 3,
 'a': 4,
 'old': 5,
 'was': 6,
 'when': 7,
 'she': 8,
 'are': 9,
 'antibodies': 10,
 'used': 11,
 'in': 12,
 'is': 13,
 'alicia': 14,
 'wouldwouldwouldwould': 15,
 'section': 16,
 's': 17,
 'much': 18,
 'long': 19,
 'with': 20,
 'can': 21,
 'taste': 22,
 'one': 23,
 'person': 24,
 'did': 25,
 'of': 26,
 'and': 27,
 'open': 28,
 'or': 29,
 'far': 30,
 'do': 31,
 'you': 32,
 'be': 33,
 'standard': 34,
 'deviation': 35,
 'brain': 36,
 'tall': 37,
 'an': 38,
 'does': 39,
 'what': 40,
 'on': 41,
 'killebrew': 42,
 'get': 43,
 'it': 44,
 'would': 45,
 'wouldwould': 46,
 'pipe': 47,
 'from': 48,
 'play': 49,
 'disease': 50,
 'cia': 51,
 'many': 52,
 'will': 53,
 'at': 54,
 '&': 55,
 'there': 56,
 'has': 57,
 'american': 58,
 'after': 59,
 'they': 60,
 'russian': 61,
 'president': 62,
 'dmitry': 63,
 'medvedev': 64,
 'chronic': 65,
 'kidney': 66,
 'being': 67,
 'lake': 68,
 'its': 69,
 'color': 70,
 'times': 71,
 'gauge': 72,
 'wire': 73,
 'this': 74

In [477]:
answersword2int

{'how': 0,
 'were': 1,
 'to': 2,
 'the': 3,
 'a': 4,
 'old': 5,
 'was': 6,
 'when': 7,
 'she': 8,
 'are': 9,
 'antibodies': 10,
 'used': 11,
 'in': 12,
 'is': 13,
 'alicia': 14,
 'wouldwouldwouldwould': 15,
 'section': 16,
 's': 17,
 'much': 18,
 'long': 19,
 'with': 20,
 'can': 21,
 'taste': 22,
 'one': 23,
 'person': 24,
 'did': 25,
 'of': 26,
 'and': 27,
 'open': 28,
 'or': 29,
 'far': 30,
 'do': 31,
 'you': 32,
 'be': 33,
 'standard': 34,
 'deviation': 35,
 'brain': 36,
 'tall': 37,
 'an': 38,
 'does': 39,
 'what': 40,
 'on': 41,
 'killebrew': 42,
 'get': 43,
 'it': 44,
 'would': 45,
 'wouldwould': 46,
 'pipe': 47,
 'from': 48,
 'play': 49,
 'disease': 50,
 'cia': 51,
 'many': 52,
 'will': 53,
 'at': 54,
 '&': 55,
 'there': 56,
 'has': 57,
 'american': 58,
 'after': 59,
 'they': 60,
 'russian': 61,
 'president': 62,
 'dmitry': 63,
 'medvedev': 64,
 'chronic': 65,
 'kidney': 66,
 'being': 67,
 'lake': 68,
 'its': 69,
 'color': 70,
 'times': 71,
 'gauge': 72,
 'wire': 73,
 'this': 74

adding the last tokens to these two dictionaries

In [478]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1
 

creating the inverse dictionary of the answerword2int dictionary

In [479]:
answersint2word = {w_i: w for w, w_i in answersword2int.items()}

In [480]:
answersint2word

{0: 'how',
 1: 'were',
 2: 'to',
 3: 'the',
 4: 'a',
 5: 'old',
 6: 'was',
 7: 'when',
 8: 'she',
 9: 'are',
 10: 'antibodies',
 11: 'used',
 12: 'in',
 13: 'is',
 14: 'alicia',
 15: 'wouldwouldwouldwould',
 16: 'section',
 17: 's',
 18: 'much',
 19: 'long',
 20: 'with',
 21: 'can',
 22: 'taste',
 23: 'one',
 24: 'person',
 25: 'did',
 26: 'of',
 27: 'and',
 28: 'open',
 29: 'or',
 30: 'far',
 31: 'do',
 32: 'you',
 33: 'be',
 34: 'standard',
 35: 'deviation',
 36: 'brain',
 37: 'tall',
 38: 'an',
 39: 'does',
 40: 'what',
 41: 'on',
 42: 'killebrew',
 43: 'get',
 44: 'it',
 45: 'would',
 46: 'wouldwould',
 47: 'pipe',
 48: 'from',
 49: 'play',
 50: 'disease',
 51: 'cia',
 52: 'many',
 53: 'will',
 54: 'at',
 55: '&',
 56: 'there',
 57: 'has',
 58: 'american',
 59: 'after',
 60: 'they',
 61: 'russian',
 62: 'president',
 63: 'dmitry',
 64: 'medvedev',
 65: 'chronic',
 66: 'kidney',
 67: 'being',
 68: 'lake',
 69: 'its',
 70: 'color',
 71: 'times',
 72: 'gauge',
 73: 'wire',
 74: 'this'

adding the end of string token to the end of every answer

In [481]:
for i in range(len(clean_answers)):
    clean_answers[i] +=' <EOS>'

In [482]:
clean_answers

['the term african in the scope of this article refers to geographical or national origins rather than racial affiliation  <EOS>',
 'from the immigration and nationality act of wouldwouldwouldwould to wouldwouldwouldwould  an estimated total of would <EOS>',
 'african immigrants in the united states come from almost all regions in africa and do not constitute a homogeneous group  <EOS>',
 'they include people from different national  linguistic  ethnic  racial  cultural and social backgrounds  <EOS>',
 'as such  african immigrants are to be distinguished from african american people  the latter of whom are descendants of mostly west and central africans who were involuntarily brought to the united states by means of the historic atlantic slave trade  <EOS>',
 'imprisonment or incarceration is a legal penalty that may be imposed by the state for the commission of a crime  <EOS>',
 'other terms used are penitentiary  correctional facility  remand centre  detention centre  and gaol or jai

translating all the questions and the answers into integers
and replacing all the words that were filtered out by <OUT>

In [483]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [484]:
questions_into_int

[[0, 97, 97, 1, 97, 2, 3, 97],
 [0, 97, 97, 1, 97, 2, 3, 97],
 [0, 97, 97, 1, 97, 2, 3, 97],
 [0, 97, 97, 1, 97, 2, 3, 97],
 [0, 97, 97, 1, 97, 2, 3, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 5, 6, 97, 97, 7, 8, 97, 97],
 [0, 5, 6, 97, 97, 7, 8, 97, 97],
 [0, 5, 6, 97, 97, 7, 8, 97, 97],
 [0, 5, 6, 97, 97, 7, 8, 97, 97],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 5, 13, 14, 12, 15],
 [0, 5, 13, 

In [485]:
answers_into_int 

[[3, 97, 97, 12, 3, 97, 26, 74, 97, 97, 2, 97, 29, 97, 97, 97, 97, 97, 97, 96],
 [48, 3, 97, 27, 97, 97, 26, 15, 2, 15, 38, 97, 97, 26, 45, 96],
 [97,
  97,
  12,
  3,
  75,
  76,
  97,
  48,
  97,
  97,
  97,
  12,
  97,
  27,
  31,
  97,
  97,
  4,
  97,
  97,
  96],
 [60, 97, 97, 48, 97, 97, 97, 97, 97, 97, 27, 97, 97, 96],
 [77,
  78,
  97,
  97,
  9,
  2,
  33,
  97,
  48,
  97,
  58,
  97,
  3,
  97,
  26,
  97,
  9,
  97,
  26,
  97,
  97,
  27,
  97,
  97,
  97,
  1,
  97,
  97,
  2,
  3,
  75,
  76,
  79,
  97,
  26,
  3,
  97,
  97,
  97,
  97,
  96],
 [97,
  29,
  97,
  13,
  4,
  97,
  97,
  80,
  81,
  33,
  97,
  79,
  3,
  97,
  82,
  3,
  97,
  26,
  4,
  97,
  96],
 [83, 97, 11, 9, 97, 97, 97, 97, 97, 97, 97, 27, 97, 29, 97, 96],
 [12, 97, 97, 97, 97, 26, 97, 97, 84, 97, 97, 96],
 [4,
  97,
  97,
  97,
  57,
  97,
  97,
  20,
  29,
  13,
  97,
  2,
  33,
  97,
  20,
  97,
  97,
  81,
  33,
  97,
  41,
  97,
  12,
  97,
  97,
  85,
  29,
  8,
  13,
  97,
  29,
  97,
  2

sorting questions and anwers by the length of questions

In [486]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])
 

In [487]:
sorted_clean_answers

[[97,
  50,
  86,
  97,
  97,
  50,
  13,
  4,
  97,
  26,
  97,
  80,
  97,
  3,
  97,
  29,
  97,
  97,
  97,
  97,
  27,
  97,
  96],
 [97,
  50,
  97,
  2,
  97,
  50,
  80,
  97,
  3,
  97,
  97,
  97,
  97,
  50,
  97,
  97,
  26,
  3,
  36,
  27,
  66,
  27,
  97,
  97,
  50,
  96],
 [3, 97, 26, 97, 50, 9, 97, 97, 97, 97, 97, 9, 3, 90, 97, 96],
 [97,
  50,
  13,
  3,
  97,
  97,
  26,
  97,
  97,
  97,
  97,
  3,
  97,
  97,
  97,
  97,
  84,
  97,
  12,
  52,
  97,
  97,
  96],
 [54,
  3,
  97,
  97,
  97,
  97,
  27,
  50,
  84,
  97,
  54,
  4,
  97,
  97,
  12,
  97,
  27,
  97,
  97,
  96],
 [97,
  97,
  50,
  97,
  97,
  97,
  97,
  3,
  97,
  26,
  97,
  50,
  97,
  97,
  97,
  12,
  97,
  97,
  97,
  97,
  97,
  97,
  97,
  48,
  97,
  96],
 [56,
  13,
  97,
  97,
  97,
  41,
  97,
  97,
  79,
  97,
  97,
  97,
  78,
  77,
  97,
  97,
  97,
  27,
  97,
  26,
  97,
  96],
 [97, 86, 97, 97, 13, 38, 97, 97, 97, 12, 97, 97, 97, 97, 97, 27, 97, 96],
 [85,
  11,
  44,
  2,
  9

In [488]:
sorted_clean_questions

[[40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 97, 97, 50],
 [40, 39, 97, 97],
 [40, 39, 97, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [40, 70, 13, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 97, 1, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 4, 97, 97, 97],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11, 12],
 [0, 9, 10, 11,

creating placeholders for the inputs and the targets

In [489]:
# Creating placeholders for the inputs and the targets
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'target')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob
 
# Preprocessing the targets
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets
 
# Creating the Encoder RNN
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state
 
# Decoding the training set
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)
 
# Decoding the test/validation set
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions
 
# Creating the Decoder RNN
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions
 
# Building the seq2seq model
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions
 
 

In [462]:
########## PART 3 - TRAINING THE SEQ2SEQ MODEL ########## 
 
# Setting the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5
 
# Defining a session
tf.reset_default_graph()
session = tf.InteractiveSession()
 
# Loading the model inputs
inputs, targets, lr, keep_prob = model_inputs()
 
# Setting the sequence length
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')
 
# Getting the shape of the inputs tensor
input_shape = tf.shape(inputs)
 
# Getting the training and test predictions
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)
 
# Setting up the Loss Error, the Optimizer and Gradient Clipping
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)
 
# Padding the sequences with the <PAD> token
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]
 
# Splitting the data into batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch
 
# Splitting the questions and answers into training and validation sets
training_validation_split = int(len(sorted_clean_questions) * 0.15)
training_questions = sorted_clean_questions[training_validation_split:]
training_answers = sorted_clean_answers[training_validation_split:]
validation_questions = sorted_clean_questions[:training_validation_split]
validation_answers = sorted_clean_answers[:training_validation_split]
 
# Training
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 1000
checkpoint = "chatbot_weights.ckpt" # For Windows users, replace this line of code by: checkpoint = "./chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry I do not speak better, I need to practice more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print("My apologies, I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")

AttributeError: module 'tensorflow.contrib.seq2seq' has no attribute 'prepare_attention'